In [53]:
import pymysql
from datetime import datetime
from elasticsearch import Elasticsearch
from tqdm.notebook import tqdm
import json

In [54]:
def get_json(path):
    with open(path,"r",encoding="utf8") as f:
        text = f.read()
    text = text[2:]
    text = f"[{text}]"
    datas = json.loads(text)
    return datas

In [55]:
# datas = get_json("./data_files/20230918/kisti_patent_0.json")
# datas = get_json("./data_files/20231023/jobplanet_0.json")
datas = get_json("./data_files/20231031/naver_news_0.json")

In [56]:
datas = list(filter(lambda o: o is not None, datas))

In [57]:
with open("./envs.json","r",encoding="utf8") as f:
    envs = json.load(f)

# connect ES

In [58]:
es = Elasticsearch(
    hosts=envs['es_host'],
    port=envs['es_port'],
    http_auth=(envs['es_user'],envs['es_password']),
    timeout=envs['es_timeout'],
    max_retries=envs['es_max_retries']
)

In [59]:
err_list = []
sucess_li = []
n = 0

In [60]:
for data in tqdm(datas, desc="엘라스틱 적재중"):
    try:
        es.index(index="source_data", document=data)
        sucess_li.append(data['BusinessNum'])
    except Exception as e:
        err_list.append((data, e))

엘라스틱 적재중:   0%|          | 0/323 [00:00<?, ?it/s]

In [61]:
len(err_list)

0

In [62]:
es.search(
    index="source_data",
    size=0,
    query={
        "bool":{
            "must":[
                {"match":{"DataType":"naver_trend"}}
            ],
            "filter":[
                {"range":{"SearchDate":{"gte":"2023-04-01","format":"yyyy-MM-dd"}}}
            ]
        }
    },
    track_total_hits=True
)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 119851, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [63]:
# sucess_li_str = ", ".join([f"'{item}'" for item in sucess_li])    # 리스트 데이터를 string형식으로 변환
sucess_li = tuple(set(int(item) for item in sucess_li)) # set으로 중복값 제거 후 리스트 튜플로 변환
sucess_li

(1048137225,
 1058767627,
 2648136208,
 5598700560,
 1198145940,
 1058779797,
 2148889878,
 2208145181,
 1178129702,
 1728800680,
 3148637230,
 3978700206,
 2208115635,
 1068177204,
 1968600116,
 5028180149,
 8998700212,
 5058152701,
 1448106429,
 1318180286,
 7758101311,
 2208113603,
 2118886469,
 6758600133,
 5048162507,
 1188122060,
 1058793294,
 2018157518,
 1118128082,
 5688601173,
 2158693463,
 2148888281,
 1208602330,
 1058748635,
 1058782685,
 5348800350,
 1058744546,
 1028140272,
 3128103025,
 1058779900,
 1388148477,
 6208600318)

# connect mysql

In [64]:
# file_name = 'jobplanet_statistic'
# file_name = 'jobplanet_review'
# file_name = 'jobplanet_premium'

file_name = datas[0]['DataType']

In [65]:
def get_mysql_conn(my_host, my_user, my_passwd, my_database, my_connect_timeout):
    conn = None
    try:
        conn = pymysql.connect(
            host=my_host,
            user=my_user,
            passwd=my_passwd,
            database=my_database,
            connect_timeout=my_connect_timeout,
        )
    except Exception as e:
        print(e)
    return conn

In [66]:
# conn = get_mysql_conn(
#     my_host=envs.get("db_host"),
#     my_user=envs.get("db_user"),
#     my_passwd=envs.get("db_password"),
#     my_database=envs.get("db_database"),
#     my_connect_timeout=envs.get("db_connect_timeout"),
# )

conn = get_mysql_conn(
    my_host=envs.get("db_host2"),
    my_user=envs.get("db_user2"),
    my_passwd=envs.get("db_password2"),
    my_database=envs.get("db_database2"),
    my_connect_timeout=envs.get("db_connect_timeout"),
)

In [67]:
data_status = datetime.now().strftime("%Y%m%d")
sql = f"UPDATE SOURCE_DATA_STATUS SET {file_name} = {data_status} WHERE BIZ_NO IN {sucess_li}"

In [68]:
try : 
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()
except : 
    pass